In [1]:
import pandas as pd
import openpyxl
import numpy as np

In [2]:
# Ler o arquivo sem carregar nenhuma planilha especificamente para obter os nomes das planilhas
xls = pd.ExcelFile('Planilha Monitoramento_PPI - Carregamento.xlsx')

# Listar todas as planilhas disponíveis no arquivo
sheet_names = xls.sheet_names
sheet_names

['INFORMAÇÕES (1)',
 'PER (1)',
 'PER (1) - BI',
 'PLAN_EXEC ATÉ ANO ANTERIOR (1)',
 'PLAN_EXEC ATÉ ANO ANTERIOR - BI',
 'META(1)',
 'META (1) - BI',
 'EXECUTADO ANO ATUAL (pendências',
 'EXECUTADO ANO ATUAL (pendê - BI',
 'À EXECUTAR (1)',
 'À EXECUTAR (1) - BI',
 'RISCOS - Não Preencher',
 'INVESTIMENTOS - Não Preencher',
 'AUXILIAR',
 'MODELO INICIAL - Não Preencher',
 'RESUMO - Não Preencher',
 'OUTROS',
 'PER (1) - BI (2)']

In [3]:
file_path = 'Planilha Monitoramento_PPI - Carregamento.xlsx'

In [4]:
# Carregar as planilhas

df_informacoes = pd.read_excel(file_path, sheet_name='INFORMAÇÕES (1)')
df_per = pd.read_excel(file_path, sheet_name='PER (1) - BI')
df_plan_exec_ano_ant = pd.read_excel(file_path, sheet_name='PLAN_EXEC ATÉ ANO ANTERIOR - BI')
df_meta = pd.read_excel(file_path, sheet_name='META (1) - BI')
df_exec_ano_at = pd.read_excel(file_path, sheet_name='EXECUTADO ANO ATUAL (pendê - BI')
df_a_executar = pd.read_excel(file_path, sheet_name='À EXECUTAR (1) - BI')

In [5]:
# Criar cópias dos DataFrames
df_informacoes_copy = df_informacoes.copy()
df_per_copy = df_per.copy()
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant.copy()
df_meta_copy = df_meta.copy() 
df_exec_ano_at_copy = df_exec_ano_at.copy()
df_a_executar_copy = df_a_executar.copy()

<h1><b><i>INFORMAÇÕES(1)</b></i></h1>

In [6]:
#CHECKPOINT
df_informacoes_copy = df_informacoes.copy()

In [7]:
df_informacoes_copy = pd.read_excel(file_path, sheet_name='INFORMAÇÕES (1)', header=2)
df_informacoes_copy.columns

Index(['Região', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR             (Grupo Controlador)',
       'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'Nº CONTRATO',
       'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)', 'CAPEX (BI)',
       'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 'TRECHO', 'KM Inicial',
       'KM Final', 'EXTENSÃO\n (km)', 'Nº CONTRATO DE SUPERVISÃO',
       'SUPERVISORA'],
      dtype='object')

In [8]:
df_informacoes_copy = df_informacoes_copy.drop([0, 1, 2], axis = 0)
df_informacoes_copy = df_informacoes_copy.reset_index(drop=True)

In [9]:
df_informacoes_copy = df_informacoes_copy.dropna(subset=['BR'])

In [10]:
df_informacoes_copy['Etapa'] = None
df_informacoes_copy['Descrição do Trecho'] = None

for index, row in df_informacoes_copy.iterrows():
    if 'ETAPA' in str(row['TRECHO']):
        etapa_atual = row['TRECHO']
    df_informacoes_copy.at[index, 'Etapa'] = etapa_atual
    if pd.notna(row['TRECHO']) and 'ETAPA' not in str(row['TRECHO']):
        df_informacoes_copy.at[index, 'Descrição do Trecho'] = row['TRECHO']

In [11]:
df_informacoes_copy['Situação'] = df_informacoes_copy['KM Inicial'].apply(lambda x: x if str(x) in ['ATIVO', 'CADUCIDADE', 'JUDICIAL', 'RELICITAÇÃO', 'RENEGOCIAÇÃO'] else pd.NA)

# Agora preenchemos para baixo até encontrar o próximo valor não-NaN (isto é, a próxima 'Situação')
df_informacoes_copy['Situação'] = df_informacoes_copy['Situação'].fillna(method='ffill')

# Agora, separemos os valores numéricos para uma nova coluna 'KM Inicial Num'
df_informacoes_copy['KM Inicial Num'] = df_informacoes_copy['KM Inicial'].apply(lambda x: x if isinstance(x, float) else pd.NA)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_3816\2058299131.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_informacoes_copy['Situação'] = df_informacoes_copy['Situação'].fillna(method='ffill')


In [12]:
df_informacoes_copy.drop('KM Inicial', axis=1, inplace=True)

In [13]:
df_informacoes_copy.drop('TRECHO', axis=1, inplace=True)

In [14]:
df_informacoes_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [15]:
df_informacoes_copy['PROPONENTE'] = 'ANTT'

In [16]:
# Renomear Coluna ['EXECUTOR...']
# DROP de Colunas e Realocação.

df_informacoes_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_informacoes_copy = df_informacoes_copy[['Região', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'ANO LEILÃO', 'Nº CONTRATO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO',
       'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 
       'Etapa', 'Descrição do Trecho',
       'Situação', 'KM Inicial Num', 'KM Final', 'EXTENSÃO\n (km)']]

In [17]:
df_informacoes_copy.dtypes

Região                            object
SETOR                             object
UF                                object
ESTADO/LOTE                       object
BR                               float64
EMPREENDIMENTO                    object
PROPONENTE                        object
EXECUTOR\n(Grupo Controlador)     object
ESTRUTURADOR DO PROJETO           object
ANO LEILÃO                       float64
Nº CONTRATO                       object
DATA DE INÍCIO                    object
ANO DA CONCESSÃO                 float64
PRAZO (anos)                     float64
CAPEX (BI)                        object
OPEX (BI)                         object
INVESTIMENTO TOTAL (BI)           object
Etapa                             object
Descrição do Trecho               object
Situação                          object
KM Inicial Num                    object
KM Final                          object
EXTENSÃO\n (km)                   object
dtype: object

In [18]:
df_informacoes_copy.columns

Index(['Região', 'SETOR', 'UF', 'ESTADO/LOTE', 'BR', 'EMPREENDIMENTO',
       'PROPONENTE', 'EXECUTOR\n(Grupo Controlador)',
       'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'Nº CONTRATO',
       'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)', 'CAPEX (BI)',
       'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 'Etapa', 'Descrição do Trecho',
       'Situação', 'KM Inicial Num', 'KM Final', 'EXTENSÃO\n (km)'],
      dtype='object')

In [19]:
# Converter colunas para 'category'
df_informacoes_copy['SETOR'] = df_informacoes_copy['SETOR'].astype('category')
df_informacoes_copy['UF'] = df_informacoes_copy['UF'].astype('category')
df_informacoes_copy['ESTADO/LOTE'] = df_informacoes_copy['ESTADO/LOTE'].astype('category')
df_informacoes_copy['PROPONENTE'] = df_informacoes_copy['PROPONENTE'].astype('category')
df_informacoes_copy['EXECUTOR\n(Grupo Controlador)'] = df_informacoes_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_informacoes_copy['ESTRUTURADOR DO PROJETO'] = df_informacoes_copy['ESTRUTURADOR DO PROJETO'].astype('category')

# Converter colunas para 'int'
df_informacoes_copy['ANO DA CONCESSÃO'] = pd.to_numeric(df_informacoes_copy['ANO DA CONCESSÃO'], errors='coerce').fillna(0).astype(int)
df_informacoes_copy['PRAZO (anos)'] = pd.to_numeric(df_informacoes_copy['PRAZO (anos)'], errors='coerce').fillna(0).astype(int)

# Converter colunas para 'float'
df_informacoes_copy['CAPEX (BI)'] = pd.to_numeric(df_informacoes_copy['CAPEX (BI)'], errors='coerce')
df_informacoes_copy['OPEX (BI)'] = pd.to_numeric(df_informacoes_copy['OPEX (BI)'], errors='coerce')
df_informacoes_copy['INVESTIMENTO TOTAL (BI)'] = pd.to_numeric(df_informacoes_copy['INVESTIMENTO TOTAL (BI)'], errors='coerce')
df_informacoes_copy['KM Final'] = pd.to_numeric(df_informacoes_copy['KM Final'], errors='coerce')
df_informacoes_copy['EXTENSÃO\n (km)'] = pd.to_numeric(df_informacoes_copy['EXTENSÃO\n (km)'], errors='coerce')

# Converter colunas para 'string'
df_informacoes_copy['EMPREENDIMENTO'] = df_informacoes_copy['EMPREENDIMENTO'].astype('string') 


In [20]:
# Função para limpar e converter as datas para o formato datetime
def limpar_e_converter_data(data):
    # Tratar dados como string para evitar problemas com diferentes tipos
    data = str(data)
    # Substituir '//'' por '/'
    data = data.replace('//', '/')
    # Substituir '/' por '-'
    data = data.replace('/', '-')
    # Tentar converter para datetime
    return pd.to_datetime(data, errors='coerce', dayfirst=True)

# Aplicar a função no DataFrame
# Substitua 'df_informacoes_copy' pelo nome do seu DataFrame e 'DATA DE INÍCIO' pelo nome da coluna que contém as datas
df_informacoes_copy['DATA DE INÍCIO'] = df_informacoes_copy['DATA DE INÍCIO'].apply(limpar_e_converter_data)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_3816\83231978.py:10: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(data, errors='coerce', dayfirst=True)


In [22]:
columns_to_fill = ['Região', 'ESTADO/LOTE', 'EMPREENDIMENTO', 'EXECUTOR\n(Grupo Controlador)']
df_informacoes_copy[columns_to_fill] = df_informacoes_copy.groupby((df_informacoes_copy['BR'] == 1).cumsum())[columns_to_fill].transform(lambda x: x.ffill())

In [23]:
df_informacoes_copy.to_excel('INFORMAÇÕES(1).xlsx', index=False)

<h1><b><i>PER(1)</b></i></h1>

In [15]:
#CHECKPOINT
df_per_copy = df_per.copy()

In [16]:
df_per_copy

,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,PAVIMENTAÇÃO - Até 2022 - Descrição,...,TERCEIRA FAIXA - 2026 - FINANCEIRO R$),TERCEIRA FAIXA - Pós 2026 - Descrição,TERCEIRA FAIXA - Pós 2026 - km (i),TERCEIRA FAIXA - Pós 2026 - km (f),TERCEIRA FAIXA - Pós 2026 - Ext. (km),TERCEIRA FAIXA - Pós 2026 - PERCENTUAL (%),TERCEIRA FAIXA - Pós 2026 - (km)%,TERCEIRA FAIXA - Pós 2026 - FX AD. TOTAL,TERCEIRA FAIXA - Pós 2026 - REL. FÍSICO (km),TERCEIRA FAIXA - Pós 2026 - REL.FINANCEIRO R$)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAVIMENTAÇÃO,...,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA,TERCEIRA FAIXA
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Até 2022,...,2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026,Pós 2026
2,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,Descrição,...,FINANCEIRO R$),Descrição,km (i),km (f),Ext. (km),PERCENTUAL (%),(km)%,FX AD. TOTAL,REL. FÍSICO (km),REL.FINANCEIRO R$)
3,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,PER (1),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_per_copy = df_per_copy.dropna(subset=['BR'])

In [18]:
df_per_copy = df_per_copy.loc[df_per_copy['Bin'] != 'Bin']
df_per_copy = df_per_copy[df_per_copy['Bin'] != 0]
df_per_copy = df_per_copy.dropna(subset=['Bin'])

In [19]:
df_per_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [20]:
df_per_copy['PROPONENTE'] = 'ANTT'

In [11]:
colunas_manter = [
    'Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_per_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_per_copy = df_per_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')


In [12]:
df_per_copy = df_per_copy.loc[df_per_copy['Valor'] != 0]
df_per_copy = df_per_copy.reset_index(drop=True)

In [21]:
df_per_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'PAVIMENTAÇÃO - Até 2022 - Descrição',
       ...
       'TERCEIRA FAIXA - 2026 - FINANCEIRO R$)',
       'TERCEIRA FAIXA - Pós 2026 - Descrição',
       'TERCEIRA FAIXA - Pós 2026 - km (i)',
       'TERCEIRA FAIXA - Pós 2026 - km (f)',
       'TERCEIRA FAIXA - Pós 2026 - Ext. (km)',
       'TERCEIRA FAIXA - Pós 2026 - PERCENTUAL (%)',
       'TERCEIRA FAIXA - Pós 2026 - (km)%',
       'TERCEIRA FAIXA - Pós 2026 - FX AD. TOTAL',
       'TERCEIRA FAIXA - Pós 2026 - REL. FÍSICO (km)',
       'TERCEIRA FAIXA - Pós 2026 - REL.FINANCEIRO R$)'],
      dtype='object', length=273)

In [13]:
df_per_copy

,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,Atributo,Valor
0,PER (1),1,Rodoviário,MG,381,FERNÃO DIAS,ANTT,ARTERIS,EPL,CONTORNO - Até 2022 - (km)%,7.5
1,PER (1),1,Rodoviário,SC,101,LITORAL,ANTT,ARTERIS,EPL,CONTORNO - Até 2022 - (km)%,94.66
2,PER (1),1,Rodoviário,PR,116,REGIS,ANTT,ARTERIS,EPL,CONTORNO - Até 2022 - (km)%,23.57
3,PER (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,CONTORNO - Até 2022 - (km)%,4.96
4,PER (1),1,Rodoviário,RJ,101,FLUMINENSE,ANTT,ARTERIS,EPL,CONTORNO - Até 2022 - (km)%,4.8
...,...,...,...,...,...,...,...,...,...,...,...
9602,PER (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,TERCEIRA FAIXA - Até 2022 - km (i),99.8
9603,PER (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,TERCEIRA FAIXA - Até 2022 - km (i),161.8
9604,PER (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,TERCEIRA FAIXA - Até 2022 - km (i),182.7
9605,PER (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,TERCEIRA FAIXA - Até 2022 - km (i),255.4


In [31]:
df_per_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_per_copy['Atributo'].str.split('-', expand=True)


In [32]:
# Renomear Coluna ['EXECUTOR...']
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_per_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_per_copy = df_per_copy[['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [33]:
df_per_copy = df_per_copy.reset_index(drop=True)

In [34]:
df_per_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor'],
      dtype='object')

In [35]:
df_per_copy.dtypes

Planilha                         object
Bin                              object
Setor                            object
UF                               object
BR                               object
EMPREENDIMENTO                   object
PROPONENTE                       object
EXECUTOR\n(Grupo Controlador)    object
ESTRUTURADOR DO PROJETO          object
Atributo.1                       object
Atributo.2                       object
Atributo.3                       object
Valor                            object
dtype: object

In [36]:
# Converter colunas para 'category'
df_per_copy['Planilha'] =  df_per_copy['Planilha'].astype('category')
df_per_copy['Bin'] =  df_per_copy['Bin'].astype(int)
df_per_copy['Setor'] =  df_per_copy['Setor'].astype('category')
df_per_copy['UF'] =  df_per_copy['UF'].astype('category')
df_per_copy['PROPONENTE'] =  df_per_copy['PROPONENTE'].astype('category')
df_per_copy['EXECUTOR\n(Grupo Controlador)'] =  df_per_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_per_copy['ESTRUTURADOR DO PROJETO'] =  df_per_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_per_copy['Atributo.1'] =  df_per_copy['Atributo.1'].astype('category')
df_per_copy['Atributo.3'] =  df_per_copy['Atributo.3'].astype('category')

# Converter colunas para 'string'
df_per_copy['EMPREENDIMENTO'] =  df_per_copy['EMPREENDIMENTO'].astype('string') 


In [37]:
df_per_copy['Planilha'] = 'PER(1)'

In [38]:
df_per_copy.to_excel('PER(1).xlsx', index=False)

<h1><b><i>PLAN_EXEC_ATÉ_ANO_ANTERIOR(1)</b></i></h1>

In [39]:
#CHECKPOINT
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant.copy()

In [40]:
df_plan_exec_ano_ant_copy.dropna(subset=['BR'])

,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,PAVIMENTAÇÃO (Acumulada até ano anterior) - Descrição,...,TERCEIRA FAIXA (Acumulada até ano anterior) - Descrição,TERCEIRA FAIXA (Acumulada até ano anterior) - km (i),TERCEIRA FAIXA (Acumulada até ano anterior) - km (f),TERCEIRA FAIXA (Acumulada até ano anterior) - Ext. (km),TERCEIRA FAIXA (Acumulada até ano anterior) - PERCENTUAL (%) PLAN,TERCEIRA FAIXA (Acumulada até ano anterior) - (km)% PLAN,TERCEIRA FAIXA (Acumulada até ano anterior) - FINANCEIRO PLAN (R$),TERCEIRA FAIXA (Acumulada até ano anterior) - PERCENTUAL (%) EXEC,TERCEIRA FAIXA (Acumulada até ano anterior) - (km)% EXEC,TERCEIRA FAIXA (Acumulada até ano anterior) - FINANCEIRO (R$)
1,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,Descrição,...,Descrição,km (i),km (f),Ext. (km),PERCENTUAL (%) PLAN,(km)% PLAN,FINANCEIRO PLAN (R$),PERCENTUAL (%) EXEC,(km)% EXEC,FINANCEIRO (R$)
10,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RS,101,CONCESSÃO VIA SUL,ANTT,CCR,EPL,0,...,0,0,0,0,0,0,0,0,0,0
11,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RS,101,CONCESSÃO VIA SUL,ANTT,CCR,EPL,0,...,0,0,0,0,0,0,0,0,0,0
13,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RS,290,CONCESSÃO VIA SUL,ANTT,CCR,EPL,0,...,0,0,0,0,0,0,0,0,0,0
14,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RS,290,CONCESSÃO VIA SUL,ANTT,CCR,EPL,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,MT,406,CRO,ANTT,ODEBRECHT,EPL,0,...,0,0,0,0,0,0,0,0,0,0
391,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,MT,406,CRO,ANTT,ODEBRECHT,EPL,0,...,0,0,0,0,0,0,0,0,0,0
400,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RJ,101,ECO PONTE,ANTT,ECORODOVIAS,EPL,Execução de Alça de Ligação Ponte - Linha Verm...,...,0,0,0,0,0,0,0,0,0,0
401,PLAN_EXEC ATÉ ANO ANTERIOR (1),1,Rodoviário,RJ,101,ECO PONTE,ANTT,ECORODOVIAS,EPL,Execução de Ligação entre a Ponte e a Avenida ...,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.loc[df_plan_exec_ano_ant_copy['Bin'] != 'Bin']
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[df_plan_exec_ano_ant_copy['Bin'] != 0]
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.dropna(subset=['Bin'])

In [42]:
df_plan_exec_ano_ant_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [43]:
df_plan_exec_ano_ant_copy['PROPONENTE'] = 'ANTT'

In [44]:
colunas_manter = [
    'Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_plan_exec_ano_ant_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [45]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.loc[df_plan_exec_ano_ant_copy['Valor'] != 0]
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.reset_index(drop=True)

In [46]:
df_plan_exec_ano_ant_copy[['Atributo.1', 'Atributo.2']] = df_plan_exec_ano_ant_copy['Atributo'].str.split('-', expand=True)

In [47]:
# Renomear Coluna ['EXECUTOR...']
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_plan_exec_ano_ant_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy[['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Valor']]

In [48]:
df_plan_exec_ano_ant_copy['Atributo.1'] = df_plan_exec_ano_ant_copy['Atributo.1'].apply(
    lambda x: x.replace('CONTORNO (Acumulada até ano anterior) ', 'CONTORNO')
              .replace('DUPLICAÇÃO (Acumulada até ano anterior) ', 'DUPLICAÇÃO')
              .replace('FX ADICIONAL (Acumulada até ano anterior) ', 'FX ADICIONAL')
              .replace('OAE (Acumulada até ano anterior) ', 'OAE')
              .replace('PAVIMENTAÇÃO (Acumulada até ano anterior) ', 'PAVIMENTAÇÃO')
              .replace('TERCEIRA FAIXA (Acumulada até ano anterior) ', 'TERCEIRA FAIXA')
)

In [49]:
df_plan_exec_ano_ant_copy = df_plan_exec_ano_ant_copy.reset_index(drop=True)

In [50]:
df_plan_exec_ano_ant_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'Atributo.1', 'Atributo.2', 'Valor'],
      dtype='object')

In [51]:
df_plan_exec_ano_ant_copy.dtypes

Planilha                         object
Bin                              object
Setor                            object
UF                               object
BR                               object
EMPREENDIMENTO                   object
PROPONENTE                       object
EXECUTOR\n(Grupo Controlador)    object
ESTRUTURADOR DO PROJETO          object
Atributo.1                       object
Atributo.2                       object
Valor                            object
dtype: object

In [52]:
# Converter colunas para 'category'
df_plan_exec_ano_ant_copy['Planilha'] = df_plan_exec_ano_ant_copy['Planilha'].astype('category')
df_plan_exec_ano_ant_copy['Bin'] = df_plan_exec_ano_ant_copy['Bin'].astype(int)
df_plan_exec_ano_ant_copy['Setor'] = df_plan_exec_ano_ant_copy['Setor'].astype('category')
df_plan_exec_ano_ant_copy['UF'] = df_plan_exec_ano_ant_copy['UF'].astype('category')
df_plan_exec_ano_ant_copy['PROPONENTE'] = df_plan_exec_ano_ant_copy['PROPONENTE'].astype('category')
df_plan_exec_ano_ant_copy['EXECUTOR\n(Grupo Controlador)'] = df_plan_exec_ano_ant_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_plan_exec_ano_ant_copy['ESTRUTURADOR DO PROJETO'] = df_plan_exec_ano_ant_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_plan_exec_ano_ant_copy['Atributo.1'] = df_plan_exec_ano_ant_copy['Atributo.1'].astype('category')
df_plan_exec_ano_ant_copy['Atributo.2'] = df_plan_exec_ano_ant_copy['Atributo.2'].astype('category')

# Converter colunas para 'string'
df_plan_exec_ano_ant_copy['EMPREENDIMENTO'] = df_plan_exec_ano_ant_copy['EMPREENDIMENTO'].astype('string') 


In [53]:
df_plan_exec_ano_ant_copy['Planilha'] = 'PLAN_EXEC_ATÉ_ANO_ANTERIOR(1)'

In [54]:
df_plan_exec_ano_ant_copy.to_excel('PLAN_EXEC_ATÉ_ANO_ANTERIOR(1).xlsx', index=False)

<h1><b><i>META(1)</b></i></h1>

In [55]:
#CHECKPOINT
df_meta_copy = df_meta.copy()

In [56]:
df_meta_copy = df_meta_copy.dropna(subset=['BR'])

In [57]:
df_meta_copy = df_meta_copy.loc[df_meta_copy['Bin'] != 'Bin']
df_meta_copy = df_meta_copy[df_meta_copy['Bin'] != 0]
df_meta_copy = df_meta_copy.dropna(subset=['Bin'])

In [58]:
df_meta_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [59]:
df_meta_copy['PROPONENTE'] = 'ANTT'

In [60]:
colunas_manter = [
    'Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_meta_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_meta_copy = df_meta_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [61]:
df_meta_copy = df_meta_copy.loc[df_meta_copy['Valor'] != 0]
#df_meta_copy = df_meta_copy.dropna(subset=['Valor'])
df_meta_copy = df_meta_copy.reset_index(drop=True)

In [62]:
df_meta_copy

,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,Atributo,Valor
0,META (1),1,Rodoviário,RS,101,CONCESSÃO VIA SUL,ANTT,CCR,EPL,Contorno - Ano Vigente - Descrição,NaN
1,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,Contorno - Ano Vigente - Descrição,NaN
2,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,Contorno - Ano Vigente - Descrição,NaN
3,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,Contorno - Ano Vigente - Descrição,NaN
4,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,Contorno - Ano Vigente - Descrição,NaN
...,...,...,...,...,...,...,...,...,...,...,...
598,META (1),1,Rodoviário,PA,163,VIA BRASIL,ANTT,CONASA,EPL,PAVIMENTAÇÃO - Ano Vigente - km (i),226
599,META (1),1,Rodoviário,PA,163,VIA BRASIL,ANTT,CONASA,EPL,PAVIMENTAÇÃO - Ano Vigente - km (i),278.84
600,META (1),1,Rodoviário,SP,116,RIO/SP,ANTT,CCR,EPL,PAVIMENTAÇÃO - Ano Vigente - km (i),151.3
601,META (1),1,Rodoviário,SP,116,RIO/SP,ANTT,CCR,EPL,PAVIMENTAÇÃO - Ano Vigente - km (i),152.89


In [64]:
df_meta_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_meta_copy['Atributo'].str.split('-', expand=True)

In [65]:
df_meta_copy['Atributo.2'].unique()

array([' Descrição', ' (km)% EXEC', ' (km)% PLAN', ' Ext. (km)',
       ' FINANANCEIRO PLAN (R$)', ' FINANCEIRO (R$)',
       ' PERCENTUAL (%) EXEC', ' PERCENTUAL (%) PLAN', ' km (f)',
       ' km (i)'], dtype=object)

In [66]:
df_meta_copy[df_meta_copy['Atributo.2'] == ' FINANANCEIRO PLAN (R$)']

,Planilha,Bin,Setor,UF,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,Atributo,Valor,Atributo.1,Atributo.3,Atributo.2
57,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,DUPLICAÇÃO - Ano Vigente - FINANANCEIRO PLAN (R$),32000000.0,DUPLICAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
58,META (1),1,Rodoviário,SP,153,TRANSBRASILIANA,ANTT,TRIUNFO,EPL,DUPLICAÇÃO - Ano Vigente - FINANANCEIRO PLAN (R$),264000000,DUPLICAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
59,META (1),1,Rodoviário,RJ,101,FLUMINENSE,ANTT,ARTERIS,EPL,DUPLICAÇÃO - Ano Vigente - FINANANCEIRO PLAN (R$),14400000.0,DUPLICAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
60,META (1),1,Rodoviário,RJ,101,FLUMINENSE,ANTT,ARTERIS,EPL,DUPLICAÇÃO - Ano Vigente - FINANANCEIRO PLAN (R$),14400000.0,DUPLICAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
112,META (1),1,Rodoviário,RS,386,CONCESSÃO VIA SUL,ANTT,CCR,EPL,FX Adicional - Ano Vigente - FINANANCEIRO PLAN...,478400.0,FX Adicional,Ano Vigente,FINANANCEIRO PLAN (R$)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,META (1),1,Rodoviário,PA,163,VIA BRASIL,ANTT,CONASA,EPL,PAVIMENTAÇÃO - Ano Vigente - FINANANCEIRO PLAN...,2400000.0,PAVIMENTAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
496,META (1),1,Rodoviário,PA,163,VIA BRASIL,ANTT,CONASA,EPL,PAVIMENTAÇÃO - Ano Vigente - FINANANCEIRO PLAN...,3000000,PAVIMENTAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
497,META (1),1,Rodoviário,SP,116,RIO/SP,ANTT,CCR,EPL,PAVIMENTAÇÃO - Ano Vigente - FINANANCEIRO PLAN...,19170000.0,PAVIMENTAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)
498,META (1),1,Rodoviário,SP,116,RIO/SP,ANTT,CCR,EPL,PAVIMENTAÇÃO - Ano Vigente - FINANANCEIRO PLAN...,12420000.0,PAVIMENTAÇÃO,Ano Vigente,FINANANCEIRO PLAN (R$)


In [67]:
# Renomear Coluna ['EXECUTOR...']
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_meta_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_meta_copy = df_meta_copy[['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [68]:
df_meta_copy = df_meta_copy.reset_index(drop=True)

In [69]:
df_meta_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor'],
      dtype='object')

In [70]:
df_meta_copy.dtypes

Planilha                         object
Bin                              object
Setor                            object
UF                               object
BR                               object
EMPREENDIMENTO                   object
PROPONENTE                       object
EXECUTOR\n(Grupo Controlador)    object
ESTRUTURADOR DO PROJETO          object
Atributo.1                       object
Atributo.2                       object
Atributo.3                       object
Valor                            object
dtype: object

In [71]:
# Converter colunas para 'category'
df_meta_copy['Planilha'] =  df_meta_copy['Planilha'].astype('category')
df_meta_copy['Bin'] =  df_meta_copy['Bin'].astype(int)
df_meta_copy['Setor'] =  df_meta_copy['Setor'].astype('category')
df_meta_copy['UF'] =  df_meta_copy['UF'].astype('category')
df_meta_copy['PROPONENTE'] =  df_meta_copy['PROPONENTE'].astype('category')
df_meta_copy['EXECUTOR\n(Grupo Controlador)'] =  df_meta_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_meta_copy['ESTRUTURADOR DO PROJETO'] =  df_meta_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_meta_copy['Atributo.1'] =  df_meta_copy['Atributo.1'].astype('category')
df_meta_copy['Atributo.2'] = df_meta_copy['Atributo.2'].astype('category')
df_meta_copy['Atributo.3'] =  df_meta_copy['Atributo.3'].astype('category')

# Converter colunas para 'string'
df_meta_copy['EMPREENDIMENTO'] =  df_meta_copy['EMPREENDIMENTO'].astype('string') 

In [72]:
df_meta_copy['Planilha'] = 'META(1)'

In [73]:
df_meta_copy.to_excel('META(1).xlsx', index=False)

<h1><b><i>EXECUTADO_ANO_ATUAL(PENDÊNCIAS)</b></i></h1>

In [74]:
#CHECKPOINT
df_exec_ano_at_copy = df_exec_ano_at.copy()

In [75]:
df_exec_ano_at_copy = df_exec_ano_at_copy.dropna(subset=['BR'])

In [76]:
df_exec_ano_at_copy = df_exec_ano_at_copy.loc[df_exec_ano_at_copy['Bin'] != 'Bin']
df_exec_ano_at_copy = df_exec_ano_at_copy[df_exec_ano_at_copy['Bin'] != 0]
df_exec_ano_at_copy = df_exec_ano_at_copy.dropna(subset=['Bin'])

In [77]:
df_exec_ano_at_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [78]:
df_exec_ano_at_copy['PROPONENTE'] = 'ANTT'

In [79]:
colunas_manter = [
    'Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_exec_ano_at_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_exec_ano_at_copy = df_exec_ano_at_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [80]:
df_exec_ano_at_copy = df_exec_ano_at_copy.loc[df_exec_ano_at_copy['Valor'] != 0]
df_exec_ano_at_copy = df_exec_ano_at_copy.reset_index(drop=True)

In [81]:
df_exec_ano_at_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_exec_ano_at_copy['Atributo'].str.extract(r'(.*?)\s*\((.*?)\)\s*-\s*(.*)')


In [82]:
# Renomear Coluna ['EXECUTOR...']
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_exec_ano_at_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_exec_ano_at_copy = df_exec_ano_at_copy[['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [83]:
df_exec_ano_at_copy = df_exec_ano_at_copy.reset_index(drop=True)

In [84]:
df_exec_ano_at_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor'],
      dtype='object')

In [85]:
df_exec_ano_at_copy.dtypes

Planilha                         object
Bin                              object
Setor                            object
UF                               object
BR                               object
EMPREENDIMENTO                   object
PROPONENTE                       object
EXECUTOR\n(Grupo Controlador)    object
ESTRUTURADOR DO PROJETO          object
Atributo.1                       object
Atributo.2                       object
Atributo.3                       object
Valor                            object
dtype: object

In [86]:
# Converter colunas para 'category'
df_exec_ano_at_copy['Planilha'] =  df_exec_ano_at_copy['Planilha'].astype('category')
df_exec_ano_at_copy['Bin'] =  df_exec_ano_at_copy['Bin'].astype(int)
df_exec_ano_at_copy['Setor'] =  df_exec_ano_at_copy['Setor'].astype('category')
df_exec_ano_at_copy['UF'] =  df_exec_ano_at_copy['UF'].astype('category')
df_exec_ano_at_copy['PROPONENTE'] =  df_exec_ano_at_copy['PROPONENTE'].astype('category')
df_exec_ano_at_copy['EXECUTOR\n(Grupo Controlador)'] =  df_exec_ano_at_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_exec_ano_at_copy['ESTRUTURADOR DO PROJETO'] =  df_exec_ano_at_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_exec_ano_at_copy['Atributo.1'] =  df_exec_ano_at_copy['Atributo.1'].astype('category')
df_exec_ano_at_copy['Atributo.3'] =  df_exec_ano_at_copy['Atributo.3'].astype('category')

# Converter colunas para 'string'
df_exec_ano_at_copy['EMPREENDIMENTO'] =  df_exec_ano_at_copy['EMPREENDIMENTO'].astype('string') 


In [87]:
df_exec_ano_at_copy['Planilha'] = 'EXECUTADO_ANO_ATUAL(PENDÊNCIAS)'
df_exec_ano_at_copy['Atributo.2'] = '(Jan-Mês Presente)'

In [88]:
df_exec_ano_at_copy.to_excel('EXECUTADO_ANO_ATUAL(PENDÊNCIAS).xlsx', index=False)

<h1><b><i>À_EXECUTAR(1)</i></b></h1>

In [89]:
#CHECKPOINT
df_a_executar_copy = df_a_executar.copy()

In [90]:
df_a_executar_copy = df_a_executar_copy.dropna(subset=['BR'])

In [91]:
df_a_executar_copy = df_a_executar_copy.loc[df_a_executar_copy['Bin'] != 'Bin']
df_a_executar_copy = df_a_executar_copy[df_a_executar_copy['Bin'] != 0]
df_a_executar_copy = df_a_executar_copy.dropna(subset=['Bin'])

In [92]:
df_a_executar_copy['ESTRUTURADOR DO PROJETO'] = 'EPL'

In [93]:
df_a_executar_copy['PROPONENTE'] = 'ANTT'

In [94]:
colunas_manter = [
    'Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR             (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
]

# Use 'difference' para obter as colunas a serem unpivotadas
colunas_unpivot = df_a_executar_copy.columns.difference(colunas_manter)

# Realizar o unpivot (melt) das outras colunas
df_a_executar_copy = df_a_executar_copy.melt(id_vars=colunas_manter, value_vars=colunas_unpivot,
                    var_name='Atributo', value_name='Valor')

In [95]:
df_a_executar_copy = df_a_executar_copy.loc[df_a_executar_copy['Valor'] != 0]
df_a_executar_copy = df_a_executar_copy.reset_index(drop=True)

In [96]:
df_a_executar_copy =df_a_executar_copy[df_a_executar_copy['Bin'] != 0]

In [97]:
df_a_executar_copy[['Atributo.1', 'Atributo.3', 'Atributo.2']] = df_a_executar_copy['Atributo'].str.split('-', expand=True)

In [98]:
# Renomear Coluna ['EXECUTOR...']
# DROP de ['Atributo'] e Realocar ['Valor'] para o final da Tabela.

df_a_executar_copy.rename(columns={'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)'}, inplace=True)

df_a_executar_copy = df_a_executar_copy[['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO'
       , 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor']]

In [99]:
df_a_executar_copy.columns

Index(['Planilha', 'Bin', 'Setor', 'UF', 'BR', 'EMPREENDIMENTO', 'PROPONENTE',
       'EXECUTOR\n(Grupo Controlador)', 'ESTRUTURADOR DO PROJETO',
       'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor'],
      dtype='object')

In [100]:
df_a_executar_copy.dtypes

Planilha                         object
Bin                              object
Setor                            object
UF                               object
BR                               object
EMPREENDIMENTO                   object
PROPONENTE                       object
EXECUTOR\n(Grupo Controlador)    object
ESTRUTURADOR DO PROJETO          object
Atributo.1                       object
Atributo.2                       object
Atributo.3                       object
Valor                            object
dtype: object

In [101]:
# Converter colunas para 'category'
df_a_executar_copy['Planilha'] =  df_a_executar_copy['Planilha'].astype('category')
df_a_executar_copy['Bin'] =  df_a_executar_copy['Bin'].astype(int)
df_a_executar_copy['Setor'] =  df_a_executar_copy['Setor'].astype('category')
df_a_executar_copy['UF'] =  df_a_executar_copy['UF'].astype('category')
df_a_executar_copy['PROPONENTE'] =  df_a_executar_copy['PROPONENTE'].astype('category')
df_a_executar_copy['EXECUTOR\n(Grupo Controlador)'] =  df_a_executar_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
df_a_executar_copy['ESTRUTURADOR DO PROJETO'] =  df_a_executar_copy['ESTRUTURADOR DO PROJETO'].astype('category')
df_a_executar_copy['Atributo.1'] =  df_a_executar_copy['Atributo.1'].astype('category')
df_a_executar_copy['Atributo.3'] =  df_a_executar_copy['Atributo.3'].astype('category')

# Converter colunas para 'string'
df_a_executar_copy['EMPREENDIMENTO'] =  df_a_executar_copy['EMPREENDIMENTO'].astype('string') 


In [102]:
df_a_executar_copy['Planilha'] = 'À_EXECUTAR(1)'

In [103]:
df_a_executar_copy.to_excel('À_EXECUTAR(1).xlsx', index=False)